## OpenAI Fine-Tune

### 生成测试数据

In [ ]:
import os,openai,backoff
import pandas as pd

openai.api_key = os.getenv("OPENAI_API_KEY")
dynasties= ['唐', '宋', '元', '明', '清', '汉', '魏', '晋', '南北朝']
super_powers = ['隐形', '飞行', '读心术', '瞬间移动', '不死之身', '喷火']
story_types = ['轻松', '努力', '艰难']

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def gpt35(prompt, max_tokens=2048, temperature=0.5, top_p=1, frequency_penalty=0, presence_penalty=0):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty)
    return response["choices"][0]["text"]

def prepare_stories(dynasties, super_powers, story_types, output_file="data/ultraman_stories.csv"):
    df = pd.DataFrame()
    repeat = 3
    for dynasty in dynasties:
        for super_power in super_powers:
            for story_type in story_types:
                   for i in range(repeat):
                        prompt = f"""请你用中文写一段300字的故事，情节跌宕起伏，讲述一位{dynasty}朝时期的英雄人物，穿越到现代，拥有了{super_power}这样的超能力，通过{story_type}的战斗，帮助奥特曼一起打败了怪兽的故事。"""
                        story = gpt35(prompt)
                        row = {"dynasty": dynasty, "super_power": super_power, "story_type": story_type, "story": story}
                        row = pd.DataFrame([row])
                        df = pd.concat([df, row], axis=0, ignore_index=True)

    df.to_csv("data/ultraman_stories.csv")

prepare_stories(dynasties, super_powers, story_types)

In [9]:
df = pd.read_csv("data/ultraman_stories.csv")
df['sub_prompt'] = df['dynasty'] + "," + df['super_power'] + "," + df['story_type']
prepared_data = df.loc[:,['sub_prompt','story']]
prepared_data.rename(columns={'sub_prompt':'prompt', 'story':'completion'}, inplace=True)
prepared_data.to_csv('data/prepared_data_mactalk.csv',index=False)

import subprocess

subprocess.run('openai tools fine_tunes.prepare_data --file data/prepared_data_mactalk.csv --quiet'.split())

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 464 prompt-completion pairs
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. Se

CompletedProcess(args=['openai', 'tools', 'fine_tunes.prepare_data', '--file', 'data/prepared_data_mactalk.csv', '--quiet'], returncode=0)

In [15]:
subprocess.run('openai api fine_tunes.create --training_file data/prepared_data_mactalk_prepared.jsonl --model curie --suffix "mactalk_ultraman"'.split())

Upload progress: 100%|██████████| 446k/446k [00:00<00:00, 342Mit/s]
Error: HTTP code 500 from API (<html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
) (HTTP status code: 500)


CompletedProcess(args=['openai', 'api', 'fine_tunes.create', '--training_file', 'data/prepared_data_mactalk_prepared.jsonl', '--model', 'curie', '--suffix', '"mactalk_ultraman"'], returncode=1)

In [11]:
subprocess.run('openai api fine_tunes.list'.split())

{
  "data": [
    {
      "created_at": 1680576711,
      "fine_tuned_model": "curie:ft-bothub-ai:ultraman-2023-04-04-03-03-26",
      "hyperparams": {
        "batch_size": 1,
        "learning_rate_multiplier": 0.2,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-3oxkr1zBVB4fJWogJDDjQbr0",
      "model": "curie",
      "object": "fine-tune",
      "organization_id": "org-yQaCtAdY0voCWSs0QNqSLfda",
      "result_files": [
        {
          "bytes": 107785,
          "created_at": 1680577408,
          "filename": "compiled_results.csv",
          "id": "file-LuSjYlkMa6fHHRB23bnr3L1z",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 446199,
          "created_at": 1680576711,
          "filename": "data/prepared_data_prepared.jsonl",
          "id": "file-yn0Bfn

CompletedProcess(args=['openai', 'api', 'fine_tunes.list'], returncode=0)

In [16]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def write_a_story(prompt):
    response = openai.Completion.create(
        model="curie:ft-bothub-ai:ultraman-2023-04-04-03-03-26",
        prompt=prompt,
        temperature=0.7,
        max_tokens=2000,
        top_p=1,
        stop=["."])
    return response["choices"][0]["text"]

story = write_a_story("宋,发射激光,艰难 ->\n")
print(story)


宋朝时期，有一位英雄人物叫做李英雄，他拥有超凡的武功，曾经参加过多次战斗，拯救了无数的人民。

一次，李英雄突然发现自己穿越到了现代，他发现自己的超能力比起宋朝时期还要强大，他可以发射激光，可以控制天空，可以控制空气，甚至可以控制大地。

李英雄发现，现代的世界正面临着一个可怕的威胁——怪兽，他们正在毁灭人类，李英雄决定要去拯救世界，于是他和奥特曼一起出发，开始了一场艰苦的战斗。

李英雄凭借着自己的超能力，和奥特曼一起，终于战胜了怪兽，拯救了世界。他以英雄的姿态拥有了榜样，成为了人们心中的英雄。


In [17]:
story = write_a_story("秦,龙卷风,辛苦 ->\n")
print(story)


这是一个关于一位叫做林黛玉的英雄人物的传奇故事。林黛玉曾经是一个英勇的将军，他在古代早期就受到了神灵们的授予，被封为“神童”。

一次，神灵们派遣一群令人鼻颈的怪兽出征，攻击辛苦的中原人民。林黛玉毫不犹豫地指挥令人恐惧的大军，抵抗怪兽的攻击，为人民谋幸福安乐。

但是，神灵们的力量太强大了，林黛玉不得不放弃了自己的家乡，离开了家园，穿越到了现代，并获得了一种超能力——龙卷风。

林黛玉很快就感受到了现代的威力，他发现自己可以用龙卷风来抵抗怪兽，于是他决定和奥特曼一起，把怪兽一个个击败，拯救人类。

他们在一次次的激烈战斗中，最终击败了最强大的怪兽，拯救了人类，林黛玉也回到了家乡，成为了一个英雄。

林黛玉的传奇故事，令人惊叹，他的勇敢和智慧，也令人敬佩。他的故事，让人们永远不会忘记。


## Flash Attention

### Softmax Tiling

In [1]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x)) # subtract max(x) for numerical stability
    return e_x / e_x.sum()

s = np.array([0.1, 0.2, 0.3, 0.4])

print(softmax(s))


[0.21383822 0.23632778 0.26118259 0.28865141]


In [4]:
## 注意，s1和s2我们是不存储的，是QV的计算结果
s1 = np.array([0.1, 0.2])
s2 = np.array([0.3, 0.4])

## 我们只存s1和s2
m1 = np.max(s1)
m2 = np.max(s2)

print(m1)
print(m2)

m = np.max([m1, m2])

0.2
0.4


In [6]:
fx1 = np.exp(s1 - m1)
fx2 = np.exp(s2 - m2)

print(fx1)
print(fx2)

[0.90483742 1.        ]
[0.90483742 1.        ]


In [19]:
fx = [np.exp(m1-m) * np.exp(s1 - m1), np.exp(m2-m) * np.exp(s2 - m2)]
print(np.hstack(fx))

[0.74081822 0.81873075 0.90483742 1.        ]


In [22]:
lx = np.exp(m1 - m) * np.sum(np.exp(s1 - m1)) + np.exp(m2 - m) * np.sum(np.exp(s2 - m2))
print(lx)

3.464386391795659


In [24]:
result = np.hstack(fx)/lx
print(result)

[0.21383822 0.23632778 0.26118259 0.28865141]


In [25]:
print(softmax(s))

[0.21383822 0.23632778 0.26118259 0.28865141]


## Tokenzier

### Tiktoken

In [27]:
import tiktoken


embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002

encoding = tiktoken.get_encoding(embedding_encoding)
text = "Deeplearning"
print(len(encoding.encode(text)))
print(encoding.encode(text))

4
[1951, 68, 698, 3256]


In [31]:
for id in encoding.encode(text):
    print(encoding.decode([id]))


De
e
ple
arning


### 扩充中文词表的Tokenizer

In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"

from transformers import LlamaTokenizer
from transformers import LlamaTokenizer
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
import sentencepiece as spm



llama_tokenizer_dir = "./data/llama_tokenizer"
chinese_sp_model_file = "./data/chinese_sp_model/chinese_sp.model"

# load
llama_tokenizer = LlamaTokenizer.from_pretrained(llama_tokenizer_dir)
chinese_sp_model = spm.SentencePieceProcessor()
chinese_sp_model.Load(chinese_sp_model_file)

llama_spm = sp_pb2_model.ModelProto()
llama_spm.ParseFromString(llama_tokenizer.sp_model.serialized_model_proto())
chinese_spm = sp_pb2_model.ModelProto()
chinese_spm.ParseFromString(chinese_sp_model.serialized_model_proto())

# print number of tokens
print(len(llama_tokenizer),len(chinese_sp_model))
print(llama_tokenizer.all_special_tokens)
print(llama_tokenizer.all_special_ids)
print(llama_tokenizer.special_tokens_map)

## Add Chinese tokens to LLaMA tokenizer
llama_spm_tokens_set=set(p.piece for p in llama_spm.pieces)
print(len(llama_spm_tokens_set))
print(f"Before:{len(llama_spm_tokens_set)}")
for p in chinese_spm.pieces:
    piece = p.piece
    if piece not in llama_spm_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        new_p.score = 0
        llama_spm.pieces.append(new_p)
print(f"New model pieces: {len(llama_spm.pieces)}")

## Save
output_sp_dir = 'merged_tokenizer_sp'
output_hf_dir = 'merged_tokenizer_hf' # the path to save Chinese-LLaMA tokenizer
os.makedirs(output_sp_dir,exist_ok=True)
with open(output_sp_dir+'/chinese_llama.model', 'wb') as f:
    f.write(llama_spm.SerializeToString())
tokenizer = LlamaTokenizer(vocab_file=output_sp_dir+'/chinese_llama.model')

tokenizer.save_pretrained(output_hf_dir)
print(f"Chinese-LLaMA tokenizer has been saved to {output_hf_dir}")

llama_tokenizer = LlamaTokenizer.from_pretrained(llama_tokenizer_dir)
chinese_llama_tokenizer = LlamaTokenizer.from_pretrained(output_hf_dir)
print(chinese_llama_tokenizer.all_special_tokens)
print(chinese_llama_tokenizer.all_special_ids)
print(chinese_llama_tokenizer.special_tokens_map)
text='''白日依山尽，黄河入海流。欲穷千里目，更上一层楼。
The primary use of LLaMA is research on large language models, including'''
print("Test text:\n",text)
print
print(f"Tokenized by LLaMA tokenizer:{llama_tokenizer.tokenize(text)}")
print(f"Tokenized by Chinese-LLaMA tokenizer:{chinese_llama_tokenizer.tokenize(text)}")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


32000 20000
['<s>', '</s>', '<unk>']
[1, 2, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
32000
Before:32000
New model pieces: 49953
Chinese-LLaMA tokenizer has been saved to merged_tokenizer_hf
['<s>', '</s>', '<unk>']
[1, 2, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
Test text:
 白日依山尽，黄河入海流。欲穷千里目，更上一层楼。
The primary use of LLaMA is research on large language models, including
Tokenized by LLaMA tokenizer:['▁', '白', '日', '<0xE4>', '<0xBE>', '<0x9D>', '山', '<0xE5>', '<0xB0>', '<0xBD>', '，', '黄', '河', '入', '海', '流', '。', '<0xE6>', '<0xAC>', '<0xB2>', '<0xE7>', '<0xA9>', '<0xB7>', '千', '里', '目', '，', '更', '上', '一', '<0xE5>', '<0xB1>', '<0x82>', '<0xE6>', '<0xA5>', '<0xBC>', '。', '<0x0A>', 'The', '▁primary', '▁use', '▁of', '▁L', 'La', 'MA', '▁is', '▁research', '▁on', '▁large', '▁language', '▁models', ',', '▁including']
Tokenized by Chinese-LLaMA tokenizer:['▁白', '日', '依', '山', '尽', '，', '黄河', '入', '海', '流', '。', '欲', '穷', '千里', '目', '，', '更', '上

### 使用SentencePiece进行Tokenizer的训练

In [1]:
import sentencepiece as spm

# Train SentencePiece model
spm.SentencePieceTrainer.train('--input=./data/mr_fujino/mr_fujino.txt --model_prefix=m --vocab_size=2000 --model_type=bpe')

# Load SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# Test encode and decode
encoded = sp.encode_as_pieces('你好，世界！一个美好的世界')
print(encoded)
decoded = sp.decode_pieces(encoded)
print(decoded)

['▁', '你', '好', ',', '世界', '!', '一个', '美', '好', '的', '世界']
你好,世界!一个美好的世界


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=./data/mr_fujino/mr_fujino.txt --model_prefix=m --vocab_size=2000 --model_type=bpe
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./data/mr_fujino/mr_fujino.txt
  input_format: 
  model_prefix: m
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
